<a href="https://colab.research.google.com/github/DevAmbani/weights-biases/blob/main/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
!apt install python3.10-venv
!python -m venv Week4HW

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,884 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.3 [5,716 B]
Fetched 2,473 kB in 3s (867 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 121752 files and directories currently installed.)
Prep

### Install packages

In [3]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [4]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [5]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [7]:
diamonds.shape

(53940, 10)

In [8]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [10]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:37:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [11]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:37:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [12]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [13]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:37:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [14]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:37:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[103]	train-rmse:375.44970	validation-rmse:532.50220


In [15]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [16]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [17]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [18]:
# Log in to your W&B account
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
sweep_config = {
    'method': 'random',  # Can be 'grid', 'random', 'bayesian'
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.01,
            'max': 0.2
        },
        'max_depth': {
            'values': [3, 5, 7, 9]
        },
        'subsample': {
            'min': 0.6,
            'max': 0.9
        },
        'colsample_bytree': {
            'min': 0.6,
            'max': 0.9
        },
        'n_estimators': {
            'values': [50, 100, 150, 200]
        }
    }
}

In [21]:
sweep_id = wandb.sweep(sweep_config, project="xgboost_diamonds_sweep_experiments", entity='devambani')

Create sweep with ID: y3o2ro2k
Sweep URL: https://wandb.ai/devambani/xgboost_diamonds_sweep_experiments/sweeps/y3o2ro2k


In [25]:
def train():
    with wandb.init() as run:
        config = wandb.config

        params = {
            'objective': 'reg:squarederror',
            'learning_rate': config.learning_rate,
            'max_depth': int(config.max_depth),
            'subsample': config.subsample,
            'colsample_bytree': config.colsample_bytree,
            'eval_metric': 'rmse'
        }

        evals = [(dtrain, 'train'), (dtest, 'test')]
        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=evals, early_stopping_rounds=10)

        predictions = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        wandb.log({'rmse': rmse})

In [26]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: epsyyfgd with config:
wandb: 	colsample_bytree: 0.7470157333200191
wandb: 	learning_rate: 0.0931057403433692
wandb: 	max_depth: 9
wandb: 	n_estimators: 100
wandb: 	subsample: 0.6197332048089546


[0]	train-rmse:3663.79090	test-rmse:3659.13685
[1]	train-rmse:3349.38124	test-rmse:3344.38568
[2]	train-rmse:3066.36419	test-rmse:3061.30027
[3]	train-rmse:2816.88964	test-rmse:2811.37631
[4]	train-rmse:2603.99160	test-rmse:2599.50343
[5]	train-rmse:2394.21937	test-rmse:2391.97171
[6]	train-rmse:2223.30560	test-rmse:2222.55851
[7]	train-rmse:2055.83091	test-rmse:2056.82411
[8]	train-rmse:1882.25070	test-rmse:1884.63026
[9]	train-rmse:1741.47085	test-rmse:1746.49423
[10]	train-rmse:1632.98443	test-rmse:1641.26245
[11]	train-rmse:1500.26918	test-rmse:1510.64309
[12]	train-rmse:1382.55979	test-rmse:1395.45621
[13]	train-rmse:1275.92926	test-rmse:1291.39048
[14]	train-rmse:1205.84845	test-rmse:1224.89446
[15]	train-rmse:1116.79660	test-rmse:1138.26182
[16]	train-rmse:1052.52003	test-rmse:1077.10278
[17]	train-rmse:979.19071	test-rmse:1006.86589
[18]	train-rmse:914.34962	test-rmse:944.79823
[19]	train-rmse:856.90899	test-rmse:891.19205
[20]	train-rmse:816.36677	test-rmse:854.34870
[21]	trai

rmse,▁
rmse,544.30118


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8tkbgeto with config:
wandb: 	colsample_bytree: 0.8382655303390566
wandb: 	learning_rate: 0.06780090209968762
wandb: 	max_depth: 9
wandb: 	n_estimators: 50
wandb: 	subsample: 0.7989327556372321


[0]	train-rmse:3743.83083	test-rmse:3740.80294
[1]	train-rmse:3507.78901	test-rmse:3504.47497
[2]	train-rmse:3288.45864	test-rmse:3284.89117
[3]	train-rmse:3075.36175	test-rmse:3072.13196
[4]	train-rmse:2885.22407	test-rmse:2882.41024
[5]	train-rmse:2700.20276	test-rmse:2697.89538
[6]	train-rmse:2546.36484	test-rmse:2543.91019
[7]	train-rmse:2384.58762	test-rmse:2382.89670
[8]	train-rmse:2234.30032	test-rmse:2233.26862
[9]	train-rmse:2094.57074	test-rmse:2093.86309
[10]	train-rmse:1974.92406	test-rmse:1975.04059
[11]	train-rmse:1852.66844	test-rmse:1854.40146
[12]	train-rmse:1739.56780	test-rmse:1742.63415
[13]	train-rmse:1634.89070	test-rmse:1639.23659
[14]	train-rmse:1550.35754	test-rmse:1556.49487
[15]	train-rmse:1458.81826	test-rmse:1466.63244
[16]	train-rmse:1374.07394	test-rmse:1382.98343
[17]	train-rmse:1295.63101	test-rmse:1306.71889
[18]	train-rmse:1222.94397	test-rmse:1235.38895
[19]	train-rmse:1155.86146	test-rmse:1170.55865
[20]	train-rmse:1099.33440	test-rmse:1115.81298
[2

...

rmse,▁
rmse,544.32912


wandb: Agent Starting Run: gwjmbfim with config:
wandb: 	colsample_bytree: 0.7798838022576992
wandb: 	learning_rate: 0.15033991249902837
wandb: 	max_depth: 5
wandb: 	n_estimators: 100
wandb: 	subsample: 0.6240162434556568


[0]	train-rmse:3456.64791	test-rmse:3451.54173
[1]	train-rmse:2987.99579	test-rmse:2980.37733
[2]	train-rmse:2595.46750	test-rmse:2588.54425
[3]	train-rmse:2256.02512	test-rmse:2250.09851
[4]	train-rmse:1976.84532	test-rmse:1970.30844
[5]	train-rmse:1734.83739	test-rmse:1730.87910
[6]	train-rmse:1558.77486	test-rmse:1553.84641
[7]	train-rmse:1381.94640	test-rmse:1377.47120
[8]	train-rmse:1237.87494	test-rmse:1233.71252
[9]	train-rmse:1116.24643	test-rmse:1111.35454
[10]	train-rmse:1027.23972	test-rmse:1022.99415
[11]	train-rmse:944.30447	test-rmse:941.78208
[12]	train-rmse:873.93470	test-rmse:873.00649
[13]	train-rmse:816.55910	test-rmse:816.42484
[14]	train-rmse:783.32477	test-rmse:785.50380
[15]	train-rmse:740.73107	test-rmse:745.74474
[16]	train-rmse:706.80901	test-rmse:713.26085
[17]	train-rmse:676.90948	test-rmse:683.46823
[18]	train-rmse:654.84702	test-rmse:662.81757
[19]	train-rmse:635.71062	test-rmse:644.44227
[20]	train-rmse:622.77169	test-rmse:633.43227
[21]	train-rmse:611.44

rmse,▁
rmse,547.47415


wandb: Agent Starting Run: g8n4552t with config:
wandb: 	colsample_bytree: 0.8411822129957723
wandb: 	learning_rate: 0.16713020780320745
wandb: 	max_depth: 7
wandb: 	n_estimators: 200
wandb: 	subsample: 0.6579110944889554
wandb: Ctrl + C detected. Stopping sweep.


##Hyperparameter Variability and Impact


*   colsample_bytree (Feature Sampling Rate): The variability in this parameter, ranging from 0.639 to 0.892, suggests it plays a significant role in defining the model’s capability to generalize. This hyperparameter specifies the proportion of total features used in building each tree within the ensemble. The absence of a consistent pattern in its impact on RMSE underscores the complexity of its interaction with other hyperparameters, indicating a need for tailored adjustments based on specific dataset characteristics and feature correlations.

*   learning_rate (Step Size for Weight Updates): Set between approximately 0.017 and 0.16, the learning rate determines the speed at which a model adjusts its weights to minimize errors. Models with intermediate learning rates around 0.1 generally yielded the best performance, showcasing that moderate rates help balance between adequate learning time and avoiding overshooting the minima. Extremely low rates may result in a prolonged training process with little improvement per iteration, while excessively high rates might cause the learning process to converge prematurely or oscillate around the optimal weights.

*   max_depth (Tree Depth): The depth of the trees, which ranged from 3 to 9, directly impacts the model’s complexity and its ability to capture underlying patterns in the data. Shallow trees (e.g., depth of 3) tend to produce higher bias and underfitting, unable to model the data complexity adequately. In contrast, deeper trees (e.g., depth of 9) are prone to learning noise and specificities of the training data, leading to overfitting. Optimal tree depth thus needs careful calibration to balance model bias and variance, particularly in relation to the dataset size and feature interactions.

*   n_estimators (Number of Trees): The configuration varied with values like 50, 100, 150, and 200 trees. Increasing the number of trees generally enhances the model's accuracy and stability by averaging out predictions, reducing the likelihood of overfitting to some extent. However, beyond a certain threshold, additional trees do not improve performance significantly and might even increase computational complexity without substantial gains, highlighting the importance of setting this parameter in context with other model settings.

*   subsample (Data Sampling Rate for Building Trees): With values ranging from about 0.638 to 0.882, this parameter controls the fraction of data used for constructing each tree. A higher subsample rate typically increases the diversity among the trees in the model, potentially improving robustness by reducing variance. However, too high a subsample rate might lead to diminished model performance due to a decrease in training data randomness.














##Performance Analysis


*   RMSE Assessment: The root mean square error (RMSE) showcased a substantial range from about 533 to 1868. Lower RMSE scores indicate more accurate model predictions. The model configurations leading to the highest RMSE often featured imbalances, such as very low learning rates combined with insufficient numbers of trees, underlining the critical nature of appropriate hyperparameter settings.

*   The best-performing models, with the lowest RMSE scores around 533, employed more balanced configurations. These avoided extremes in any single parameter, demonstrating the effectiveness of moderate settings, particularly in terms of learning rates and the number of estimators used.



##Guidelines for Enhanced Model Tuning:



*   Strategic Depth and Complexity Management: Determining the optimal tree depth is crucial, balancing the need to capture sufficient data complexity with the risk of overfitting. This often requires adjusting based on the specific data being used, possibly through cross-validation techniques to test different depths under various conditions.

*   Optimizing Learning Dynamics: Fine-tuning the learning rate can significantly affect model performance. Incremental adjustments and employing techniques such as learning rate schedules or adaptive learning rate methods can optimize this parameter for better convergence behaviors.

*   Comprehensive Hyperparameter Optimization: Utilizing systematic approaches like grid search or random search can help in uncovering the most effective combinations of parameters. These methods allow for exploring a broad space of parameter settings, ensuring that the interdependencies between hyperparameters are adequately addressed, which is crucial for achieving the best model performance.




